In [ ]:
!pip install PyPDF2
!pip install pdfminer


In [ ]:
link='https://imsdb.com/scripts/Joker.html'

In [ ]:
import requests
import json
import re
import pandas as pd
pd.options.display.max_rows = 999

In [ ]:
r=requests.get(link,headers={'Accept':'text/html'})
from bs4 import BeautifulSoup
soup = BeautifulSoup(r, 'html.parser')
raw_text=soup.get_text()[2010:-350]

TypeError: ignored

In [ ]:
r.text

In [ ]:
r.headers['Content-Type']
r.headers.get('content-type')

'text/html; charset=UTF-8'

In [ ]:
print(raw_text)

NameError: ignored

In [ ]:
# character=re.findall(r'\S[A-Z].+\b',raw_text,flags=re.M)
character=re.findall(r'[A-Z][^a-z]+?$\s',raw_text,flags=re.M)

In [ ]:
[i.strip('\n').strip('\r').strip() for i in character]

In [ ]:
characters=pd.DataFrame(character,columns=['character']).value_counts()

In [ ]:
characters[characters>2]

In [ ]:
locations=[re.findall(r"INT\. .+\D\S\b|EXT\. .+\D\S\b",i) for i in character]

In [ ]:
locations

In [ ]:
locations_=[i for i in locations if i!=[]]

In [ ]:
locations_df=pd.DataFrame(locations_,columns=['locations'])

In [ ]:
locations_df.sort_values(by='place')

In [ ]:
locations_df['int_ext']=locations_df['locations'].str.extract(r'(INT\.|EXT\.)')
locations_df['location']=[i[5:].split('-')[0] for i in locations_df['locations']]
locations_df['time_of_day']=[i[5:].split('-')[-1] for i in locations_df['locations']]


In [ ]:
locations_df['place']=[i.split(',')[0] for i in locations_df['location']]
# locations_df['area']=[i.split(',')[1] for i in locations_df['location']]

In [ ]:
'\b^\d+\b' # scene number
'\d+\.(?=\s)' #page number


In [ ]:
locations_df['time_of_day'].value_counts()

 CONTINUOUS                                       33
 NIGHT                                            26
 MORNING                                          20
 EVENING                                          10
 LATE AFTERNOON                                    7
 DAY                                               5
 DUSK                                              4
 LATER                                             3
 AFTERNOON                                         3
 LATE NIGHT                                        2
 EARLY MORNING                                     2
 NEXT AFTERNOON                                    1
 SECONDS LATER                                     1
 MAGIC HOUR                                        1
 COUNTRYSIDE, OUTSIDE GOTHAM                       1
 EVENING                    8                      1
 NIGHT                         9                   1
 CONTINUOUS            3                           1
 DUSK            4                            

In [ ]:
[i.split(',') for i in locations_df['locations']]

[['INT. DEPT. OF HEALTH', ' OFFICE - MORNING                          1'],
 ['EXT. GOTHAM SQUARE', " MIDTOWN - KENNY'S MUSIC SHOP - DAY         2"],
 ['EXT. CORNER', ' SIDE STREET - GOTHAM SQUARE - CONTINUOUS            3'],
 ['INT. CITY BUS (PULLING OUT) - HEADING DOWNTOWN - DUSK            4'],
 ['EXT. LOWER EAST SIDE',
  ' STREETS - GOTHAM - EVENING                                     5'],
 ['INT. LOBBY', ' APARTMENT BUILDING - EVENING                        6'],
 ['INT. ELEVATOR', ' APARTMENT BUILDING - CONTINUOUS                  7'],
 ["INT. MOM'S APARTMENT", ' FRONT DOOR - EVENING                    8'],
 ["INT. MOM'S BEDROOM", ' APARTMENT - NIGHT                         9'],
 ['INT. LIVING ROOM', " MOM'S APARTMENT - NIGHT"],
 ['EXT. WAYNE TOWER', ' STREET - MIDTOWN - MORNING'],
 ['INT. RECEPTION', ' WAYNE ENTERPRISES - 25TH FLOOR - MORNING'],
 ['EXT. WAYNE TOWER', ' FRONT ENTRANCE - MORNING'],
 ["INT. HA-HA'S TALENT BOOKING", ' LOCKER ROOM - DAY'],
 ['INT. FRONT OFFICE', " HA-H

In [ ]:
import re
import PyPDF2

import pandas as pd
pd.options.display.max_rows = 999
# import camelot
import os

In [ ]:
FILE_PATH = '/content/joker_new_final.pdf'

def text_extractor(filepath):
  
  reader = PyPDF2.PdfFileReader(filepath,strict=False)
  # return [f'{reader.getPage(i).extractText()} /n' for i in range(0,reader.numPages)]
  pages=[reader.getPage(i) for i in range(0,3)]
  return [i.extractText().splitlines() for i in pages] 
joker_raw_text=text_extractor(FILE_PATH)
print(joker_raw_text)

[['JOKERan originwritten byTodd Phillips & Scott Silver1 DECEMBER 2018FINAL SHOOTING SCRIPT'], ['This story takes place in its own universe. It has no connection to any of the DC films that have come before it. We see it as a classic Warner Bros. movie. Gritty, intimate and oddly funny, the characters live in the real world and the stakes are personal.Although it is never mentioned in the film, this story takes place in the past. Let\'s call it 1981.It\'s a troubled time. The crime rate in Gotham is at record highs. A garbage strike has crippled the city for the past six weeks. And the divide between the "haves" and the "have-nots" is palpable. Dreams are beyond reach, slipping into delusions.TP/SSii'], ["OVER BLACK:HEAR LAUGHTER. The sound of a man totally cracking up.FADE IN:INT. DEPT. OF HEALTH, OFFICE - MORNINGCLOSE ON ARTHUR (30's), tears in his eyes from laughing so hard. He's trying to get it under control. His greasy, black hair hanging down over his forehead. He's wearing an o

In [ ]:
FILE_PATH = '/content/joker_new_final.pdf'

In [ ]:
joker_raw_text=text_extractor(FILE_PATH)

NameError: ignored

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(FILE_PATH, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

joker_raw_text=output_string.getvalue()

In [ ]:
print(joker_raw_text)

## Extracting Locations

In [ ]:
re.findall(r"(INT+\..+|EXT+\..+)",joker_raw_text)

In [ ]:
text='''INT. TALK SHOW SET, STAGE - CONTINUOUS

Murray glances over at his producer Gene Ufland, who's 
sitting off-camera on a director's chair by a monitor. Gene 
shrugs at him. 

MURRAY FRANKLIN

(smiles; trying to save 
the interview)

So when we talked earlier, you 
mentioned that you aren't 
political. That this look isn't a 
political statement. 

JOKER

That's right. I'm not political, 
Murray. I'm just trying to make 
people laugh.
EXT. HOME
MURRAY FRANKLIN - #2'''

In [ ]:
# re.findall(r"[A-Z]+\s.+",joker_raw_text,flags=re.M)
# re.findall(r"[A-Z]+\s.*",text,flags=re.M)
# re.findall(r"([A-Z]+.+\w)",text,flags=re.M)
not_charachters=re.findall(r"^INT.+|^EXT.+|\b.[a-z].+|\(",text,flags=re.M)

In [ ]:
characters=re.sub(r'^\s\(|INT.+\s*|^EXT.+\s*|^\n|\b.[a-z].*\s*','',joker_raw_text,flags=re.M).split('\n')


In [ ]:
characters

In [ ]:
characters_=[re.sub(r'\d\.|\d','',i,flags=re.M) for i in characters]

In [ ]:
import string

In [ ]:
pd.DataFrame([i for i in characters_ if i not in string.punctuation and 3<len(i)<20]).value_counts(ascending=True)[:20]


ARTHUR (CONT’D)      1
IMURRAY FRANKLIN      1
IJOKER                1
IISOCIAL WORKER       1
IIMURRAY FRANKLIN     1
IGENE UFLAND          1
IDR. STONER (VO)      1
IDET. BURKE           1
ICUT TO:              1
ICLOSE ON ARTHUR      1
IBLAM!                1
I, IALFRED            1
I, I(MORE)            1
HEARSCUT BACK TO:     1
HARTHUR               1
HAPPY!? ARTHUR        1
HA-HA CLOWN # (OS)    1
HA-HA CLOWN #         1
HA! ARTHUR            1
GARY (OS)             1
dtype: int64